In [1]:
# 实现1.将指定文件夹内的heic、jpg转化成webp格式，确保都在webp内保留文件的exif信息，同时实现文件体积的压缩。
# 2.读取所有webp文件的exif信息，在文件夹内形成一个完整exif信息记录文件，可以未来用于读取。
import os
import json
from PIL import Image
from PIL.ExifTags import TAGS
from pillow_heif import register_heif_opener

# 注册 HEIF 打开器
register_heif_opener()

def convert_to_webp(input_path, output_path, quality=80):
    """将图像转换为 WebP 格式并保留 EXIF 信息"""
    with Image.open(input_path) as img:
        # 获取原始 EXIF 数据
        exif_data = img.info.get('exif')
        
        # 转换为 WebP
        img.save(output_path, 'WEBP', quality=quality, exif=exif_data)

def get_exif_data(image_path):
    """获取图像的 EXIF 数据"""
    with Image.open(image_path) as img:
        exif_data = {}
        info = img._getexif()
        if info:
            for tag_id, value in info.items():
                tag = TAGS.get(tag_id, tag_id)
                if isinstance(value, bytes):
                    value = value.decode(errors='replace')
                exif_data[tag] = str(value)
        return exif_data
    return None

def process_directory(directory_path, output_directory):
    """处理目录中的所有图像文件"""
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)

    exif_records = {}

    for filename in os.listdir(directory_path):
        if filename.lower().endswith(('.heic', '.jpg', '.jpeg', '.webp')):
            input_path = os.path.join(directory_path, filename)
            output_filename = os.path.splitext(filename)[0] + '.webp'
            output_path = os.path.join(output_directory, output_filename)

            # 转换图像为 WebP
            convert_to_webp(input_path, output_path)

            # 获取并存储 EXIF 数据
            exif_data = get_exif_data(output_path)
            if exif_data:
                exif_records[output_filename] = exif_data

    # 将 EXIF 记录保存到文件
    exif_record_path = os.path.join(output_directory, 'exif_records.json')
    with open(exif_record_path, 'w', encoding='utf-8') as f:
        json.dump(exif_records, f, indent=2, ensure_ascii=False)

# 使用示例，这里还是先简单按照每个子相册单独处理，避免一口气影响所有的相册图片。
input_directory = r"..\SeanBlog\public\pictures\Albums\MounthookeValley"
output_directory = r"..\SeanBlog\public\pictures\Albums\MounthookeValley"
process_directory(input_directory, output_directory)

In [ ]:
#简单的实现读取文件夹所有图片，用blip模型分析图片内容，形成一句话描述，可以用来作为图片文件重命名。目前取了前15词
import requests
import os
from pathlib import Path

API_URL = "https://api-inference.huggingface.co/models/Salesforce/blip-image-captioning-large"
#需要替换为你的huggingface的token
headers = {"Authorization": "Bearer hf_xxxx"}

def query(filename):
    with open(filename, "rb") as f:
        data = f.read()
    response = requests.post(API_URL, headers=headers, data=data)
    return response.json()

def generate_filename(description):
    # 简化描述，只保留前15个单词，并用下划线连接
    words = description.split()[:15]
    return "_".join(words).lower()

def rename_file(old_path, new_name):
    old_path = Path(old_path)
    new_path = old_path.with_name(new_name + old_path.suffix)
    os.rename(old_path, new_path)
    return new_path

def process_directory(directory):
    image_extensions = ['.jpg', '.jpeg', '.png', '.webp', '.gif']
    for filename in os.listdir(directory):
        if any(filename.lower().endswith(ext) for ext in image_extensions):
            file_path = os.path.join(directory, filename)
            try:
                # 获取图片描述
                output = query(file_path)
                description = output[0]['generated_text']
                print(f"文件: {filename}")
                print(f"图片描述: {description}")

                # 生成新文件名
                new_filename = generate_filename(description)
                print(f"生成的新文件名: {new_filename}")

                # 重命名文件
                new_path = rename_file(file_path, new_filename)
                print(f"文件已重命名为: {new_path}")
                print("-------------------")
            except Exception as e:
                print(f"处理文件 {filename} 时出错: {str(e)}")
                print("-------------------")

# 指定目录路径，同样也要跟前面一样一个个目录去操作
directory_path = r"..\SeanBlog\public\pictures\Albums\NewZealand"

# 处理目录中的所有图片
process_directory(directory_path)

In [2]:
# 这是一个针对相册压缩成缩略图的python程序，目前需要针对指定的相册文件手动运行
# 运行模式：python generate_thumbnails.py public/pictures/Albums/NewZealand public/pictures/thumbnails/NewZealand

import os
import sys
from PIL import Image, ExifTags

def create_thumbnail(image_path, output_path, size=(500, 500)):
    with Image.open(image_path) as img:
        # 处理图片方向
        try:
            for orientation in ExifTags.TAGS.keys():
                if ExifTags.TAGS[orientation] == 'Orientation':
                    break
            exif = dict(img._getexif().items())
            if exif[orientation] == 3:
                img = img.rotate(180, expand=True)
            elif exif[orientation] == 6:
                img = img.rotate(270, expand=True)
            elif exif[orientation] == 8:
                img = img.rotate(90, expand=True)
        except (AttributeError, KeyError, IndexError):
            # 图片没有 EXIF 信息，不需要旋转
            pass

        # 计算缩放比例
        img.thumbnail((size[0], size[1]), Image.LANCZOS)
        
        # 保存时不添加白边，保持原有比例
        img.save(output_path, "WEBP", quality=95)

def process_directory(input_dir, output_dir):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for filename in os.listdir(input_dir):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.webp')):
            input_path = os.path.join(input_dir, filename)
            output_filename = os.path.splitext(filename)[0] + "_thumb.webp"
            output_path = os.path.join(output_dir, output_filename)
            create_thumbnail(input_path, output_path)
            print(f"Created thumbnail for {filename}")

if __name__ == "__main__":
    if len(sys.argv) != 3:
        print("Usage: python generate_thumbnails.py <input_directory> <output_directory>")
        sys.exit(1)

    # input_dir = sys.argv[1]
    # output_dir = sys.argv[2]
    input_dir = r"..\SeanBlog\public\pictures\Albums\MounthookeValley"
    output_dir = r"..\SeanBlog\public\pictures\thumbnails\MounthookeValley"
    process_directory(input_dir, output_dir)


Usage: python generate_thumbnails.py <input_directory> <output_directory>


SystemExit: 1

C:\Users\maoxi\AppData\Roaming\Python\Python312\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
